    Fase 1: Librerias

In [1]:
import astropy
import numpy as np
import scipy
import numpy
import pyspeckit
import sedfitter
import zipfile
import os
import re
from astropy.extern import six
#from sedfitter.filter import Filter
#from sedfitter import fit
import matplotlib.pyplot as plt
#from sklearn import preprocessing
from astropy.table import QTable,Table,Column


    Fase 2: Funciones

In [3]:
def Filterdat(lista): #filtrar datos de directorios por ".dat"
    return [name for name in lista if re.search(r'dat', name)]

def colxfil(mat): #en una tabla, o matriz, cambiar filas por columnas
    num=[i for i in range(len(mat[0]))]
    aux=[]
    mat2=[]
    for pos in num:
        aux=[]
        for lista in mat:
            aux.append(lista[pos])
        
        mat2.append(aux)
    return mat2

def norm(lista): #normalizacion de datos por preprocessing
    return preprocessing.normalize(lista)

def norm2(lista): #normalización de datos por maximos y minimos
    a=[]
    for f in lista:
        a.append((f-min(lista))/(max(lista)-min(lista)))
    return a

def unique(lista): #devuelve una lista con datos unicos, los repetidos fuera
    unicos=[]
    for i in lista:
        if i not in unicos:
            unicos.append(i)
    return unicos

def recta(punto1,punto2,evaluar): #crea valores entre 2 puntos en funcion de un conjunto externo
    x1=punto1[0]
    y1=punto1[1]
    x2=punto2[0]
    y2=punto2[1]
    if x2!=x1:
        m=(y2-y1)/(x2-x1)
    if x2==x1:
        pass
    c=y1-(m*x1)
    return [(m*x)+c for x in evaluar]
    
def integrar(datos_entrada,datos_respuesta,limite_minimo,limite_maximo,cantidad_sumas): #terminado y por probar en datos reales

    delta=(limite_maximo-limite_minimo)
    divisiones=float(delta/cantidad_sumas)
    
    
    dominio_1=float(limite_minimo)
    dominio=[dominio_1]
    aprox=len(str(cantidad_sumas))
    while round(float(dominio_1),aprox)!=round(float(limite_maximo),aprox): #funciona generando las divisiones de base n, guardado en la lista dominio.
        dominio_1+=divisiones
        dominio.append(round(dominio_1,aprox))
    #return dominio
    
    coordenadas=[]
    for ejex,ejey in zip(datos_entrada,datos_respuesta): #guarda los conjuntos de coordendas que entran.
        if ejex >= min(dominio) and ejex <= max(dominio):
            for value in dominio:
                if round(float(value))==float(ejex):
                    coordenadas.append([ejex,ejey])
    #return unique(coordenadas)
    
    coordenadas2=unique(coordenadas)
    #indices=0
    
    valores=[]
    dom=[]
    respuestas=[]
    
    for contador in range(0,len(coordenadas2)-1): #correctamente se separan las rectas, y todos los valores en x tienen respuesta en el eje y.
        datos_x=[]
        puntos=[]
        for valores in dominio:
            puntos=[coordenadas2[contador],coordenadas2[contador+1]]
            if valores>=coordenadas2[contador][0] and valores <=coordenadas2[contador+1][0]:
                datos_x.append(valores)
                
        dom.append(datos_x)
        aux=recta(puntos[0],puntos[1],datos_x)
        respuestas.append(aux)    
    #return (dom,respuestas)
    salida=[]
    for recta_x,recta_y in zip(dom,respuestas): #salida correcta.
        c=0
        aux=[]
        while c!=(len(recta_x)-1):
            base=recta_x[c+1]-recta_x[c]
            altura=recta_y[c] ###revisar si el modelo creado sirve y para cual respuesta #c+1, acercamiento superior; c, acercamiento innferior.
            aux.append(base*altura)
            c+=1
        salida.append(aux)
    #return salida
    
    integrado=0
    for lista in salida: #sumatoria final; integración.
        integrado+=sum(lista)
    return integrado

    Fase 3: Filtros (datos SPLUS) 